In [ ]:
pip install gradio


In [ ]:
import gradio as gr
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import resnet18
import torch.nn as nn

# Define the custom ResNet18 model
class Resnet18(nn.Module):
    def __init__(self, num_classes=2):
        super(Resnet18, self).__init__()
        model_resnet18 = resnet18(pretrained=True)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool
        self.features = model_resnet18.fc.in_features
        self.fc = nn.Linear(self.features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Load the saved model
model = Resnet18(num_classes=2)
#model.load_state_dict(torch.load("/content/resnet18_final_epoch (6).pth"))  # Replace with your model path
model.load_state_dict(torch.load("/content/resnet18_final_epoch (6).pth", map_location=torch.device('cpu')))

model.eval()  # Set model to evaluation mode

# Define the classes
class_names = ["Benign", "Malignant"]

# Define preprocessing for input images
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the prediction function
def predict(image):
    # Convert image to tensor
    image = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Run the model
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Get the predicted class
    predicted_class = probabilities.argmax().item()
    confidence = probabilities[predicted_class].item()

    return {class_names[0]: float(probabilities[0]), class_names[1]: float(probabilities[1])}

# Define the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Binary Classification with Custom ResNet18",
    description="Upload an image to classify it as 'Benign' or 'Malignant' using a pretrained ResNet18 model.",
)

# Launch the interface
if __name__ == "__main__":
    interface.launch()
